## Training CBoW Model


## Обучающая модель CBoW

This notebooks is a part of [AI for Beginners Curriculum](http://aka.ms/ai-beginners)


Эта тетрадь является частью [AI для начинающих Curriculum] (http://aka.ms/ai-начинающих)

In this example, we will look at training CBoW language model to get our own Word2Vec embedding space. We will use AG News dataset as the source of text.

В этом примере мы рассмотрим обучающую языковую модель CBoW, чтобы получить собственное пространство внедрения Word2Vec. Мы будем использовать набор данных AG News в качестве источника текста.

In [30]:
from tensorflow import keras
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

We will start by loading the dateset:

Начнем с загрузки набора дат:

In [1]:
ds_train, ds_test = tfds.load('ag_news_subset').values()

## CBoW Model


## Модель CBoW

CBoW learns to predict a word based on the $2N$ neighboring words. For example, when $N=1$, we will get the following pairs from the sentence *I like to train networks*: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks). Here, first word is the neighboring word used as an input, and second word is the one we are predicting.


CBoW учится предсказывать слово на основе соседних слов за 2 доллара. Например, когда $N=1$, мы получим следующие пары из предложения *Мне нравится обучать сети*: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks, networks). Здесь первое слово — это соседнее слово, используемое в качестве входных данных, а второе слово — это то, которое мы предсказываем.

To build a network to predict next word, we will need to supply neighboring word as input, and get word number as output. The architecture of CBoW network is the following:


Чтобы построить сеть для прогнозирования следующего слова, нам нужно будет указать соседнее слово в качестве входных данных и получить номер слова в качестве вывода. Архитектура сети CBoW следующая:

* Input word is passed through the embedding layer. This very embedding layer would be our Word2Vec embedding, thus we will define it separately as `embedder` variable. We will use embedding size = 30 in this example, even though you might want to experiment with higher dimensions (real word2vec has 300)


* Входное слово передается через слой встраивания. Этот самый слой встраивания будет нашим встраиванием Word2Vec, поэтому мы определим его отдельно как переменную 'embedder'. В этом примере мы будем использовать встраивание size = 30, хотя вы можете поэкспериментировать с более высокими измерениями (реальный word2vec имеет 300)
* Embedding vector would then be passed to a dense layer that will predict output word. Thus it has the `vocab_size` neurons.


* Затем вектор встраивания будет передан в плотный слой, который будет предсказывать выходное слово. Таким образом, он имеет «vocab_size» нейроны.

Embedding layer in Keras automatically knows how to convert numeric input into one-hot encoding, so that we do not have to one-hot-encode input word separately. We specify `input_length=1` to indicate that we want just one word in the input sequence - normally embedding layer is designed to work with longer sequences.


Слой встраивания в Keras автоматически знает, как преобразовать числовой ввод в одно горячее кодирование, так что нам не нужно отдельно кодировать входное слово с одним горячим кодом. Мы указываем 'input_length=1', чтобы указать, что нам нужно только одно слово во входной последовательности - обычно слой встраивания предназначен для работы с более длинными последовательностями.

For the output, if we use `sparse_categorical_crossentropy` as loss function, we would also have to provide just word numbers as expected results, without one-hot encoding.


Для выходных данных, если мы используем «sparse_categorical_crossentropy» в качестве функции потерь, нам также придется предоставить только номера слов в качестве ожидаемых результатов, без однократного горячего кодирования.

We will set `vocab_size` to 5000 to limit computations a bit. We will also define a vectorizer which we will use later. 

Мы установим 'vocab_size' на 5000, чтобы немного ограничить вычисления. Мы также определим векторизатор, который мы будем использовать позже. 

In [68]:
vocab_size = 5000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))
embedder = keras.layers.Embedding(vocab_size,30,input_length=1)

model = keras.Sequential([
    embedder,
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1, 30)             150000    
                                                                 
 dense_1 (Dense)             (None, 1, 5000)           155000    
                                                                 
Total params: 305,000
Trainable params: 305,000
Non-trainable params: 0
_________________________________________________________________


Let's initialize the vectorizer and get out the vocabulary:

Инициализируем векторизатор и выйдем из словаря:

In [69]:
def extract_text(x):
    return x['title']+' '+x['description']

vectorizer.adapt(ds_train.take(500).map(extract_text))
vocab = vectorizer.get_vocabulary()

## Preparing Training Data


## Подготовка обучающих данных

Now let's program the main function that will compute CBoW word pairs from text. This function will allow us to specify window size, and will return a set of pairs - input and output word. Note that this function can be used on words, as well as on vectors/tensors - which will allow us to encode the text, before passing it to `to_cbow` function.

Теперь давайте запрограммируем основную функцию, которая будет вычислять пары слов CBoW из текста. Эта функция позволит нам указать размер окна, и вернет набор пар - входное и выходное слово. Обратите внимание, что эта функция может быть использована как на словах, так и на векторах/тензорах - что позволит нам закодировать текст, прежде чем передать его в функцию 'to_cbow'.

In [70]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(vectorizer('I like to train networks')))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[<tf.Tensor: shape=(), dtype=int64, numpy=376>, <tf.Tensor: shape=(), dtype=int64, numpy=771>], [<tf.Tensor: shape=(), dtype=int64, numpy=3>, <tf.Tensor: shape=(), dtype=int64, numpy=771>], [<tf.Tensor: shape=(), dtype=int64, numpy=771>, <tf.Tensor: shape=(), dtype=int64, numpy=376>], [<tf.Tensor: shape=(), dtype=int64, numpy=3>, <tf.Tensor: shape=(), dtype=int64, numpy=376>], [<tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=376>], [<tf.Tensor: shape=(), dtype=int64, numpy=771>, <tf.Tensor: shape=(), dtype=int64, numpy=3>], [<tf.Tensor: shape=(), dtype=int64, numpy=376>, <tf.Tensor: shape=(), dtype=int64, numpy=3>], [<tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=3>]

Let's prepare the training dataset. We will go through all news, call `to_cbow` to get the list of word pairs, and add those pairs to `X` and `Y`. For the sake of time, we will only consider first 10k news items - you can easily remove the limitation in case you have more time to wait, and want to get better embeddings :)

Подготовим обучающий набор данных. Мы рассмотрим все новости, назовем «to_cbow», чтобы получить список пар слов, и добавим эти пары к «X» и «Y». Ради экономии времени мы рассмотрим только первые 10k новостей - вы можете легко снять ограничение, если у вас есть больше времени на ожидание, и вы хотите получить лучшие встраивания :)

In [100]:
X = []
Y = []
for i,x in zip(range(10000),ds_train.map(extract_text).as_numpy_iterator()):
    for w1, w2 in to_cbow(vectorizer(x),window_size=1):
        X.append(tf.expand_dims(w1,0))
        Y.append(tf.expand_dims(w2,0))

We will also convert that data to one dataset, and batch it for training:

Мы также преобразуем эти данные в один набор данных и пакетируем их для обучения:

In [101]:
ds = tf.data.Dataset.from_tensor_slices((X,Y)).batch(256)

Now let's do the actual training. We will use `SGD` optimizer with pretty high learning rate. You can also try playing around with other optimizers, such as `Adam`. We will train for 200 epochs to begin with - and you can re-run this cell if you want even lower loss.

Теперь давайте сделаем фактическое обучение. Мы будем использовать оптимизатор 'SGD' с довольно высокой скоростью обучения. Вы также можете попробовать поиграть с другими оптимизаторами, такими как «Адам». Мы будем тренироваться в течение 200 эпох для начала - и вы можете повторно запустить эту ячейку, если хотите еще меньшие потери.

In [102]:
model.compile(optimizer=keras.optimizers.SGD(lr=0.1),loss='sparse_categorical_crossentropy')
model.fit(ds,epochs=200)

Epoch 1/200


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


2156/2156 [==============================] - 7s 3ms/step - loss: 5.6134
Epoch 2/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.5431
Epoch 3/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.5029
Epoch 4/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4754
Epoch 5/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4548
Epoch 6/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4382
Epoch 7/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4243
Epoch 8/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4123
Epoch 9/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4019
Epoch 10/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.3926
Epoch 11/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.3843
Epoch 12/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.3767
E

## Trying out Word2Vec


## Опробование Word2Vec

To use Word2Vec, let's extract vectors corresponding to all words in our vocabulary:

Чтобы использовать Word2Vec, давайте извлечем векторы, соответствующие всем словам в нашем словаре:

In [103]:
vectors = embedder(vectorizer(vocab))
vectors = tf.reshape(vectors,(-1,30)) # we need reshape to get rid of extra dimension

Let's see, for example, how the word **Paris** is encoded into a vector:

Давайте посмотрим, например, как слово **Paris** кодируется в вектор:

In [104]:
paris_vec = embedder(vectorizer('paris'))[0]
print(paris_vec)

tf.Tensor(
[-0.13308628  0.50972325  0.00344684  0.185389   -0.03176536  0.22262476
 -0.3856765  -0.6854793   0.5185803  -0.7215402  -0.16101503  0.15622072
  0.00653811 -0.14954254  0.03379822 -0.01243829  0.27907634 -0.32538188
  0.21718933  0.31112966 -0.24142407  0.15589055  0.2915561   0.19029242
  0.08425518 -0.0941902  -0.54313695 -0.24854654  0.26196313  0.18027727], shape=(30,), dtype=float32)


It is interesting to use Word2Vec to look for synonyms. The following function will return `n` closest words to a given input. To find them, we compute the norm of $|w_i - v|$, where $v$ is the vector corresponding to our input word, and $w_i$ is the encoding of $i$-th word in the vocabulary. We then sort the array and return corresponding indices using `argsort`, and take first `n` elements of the list, which encode positions of closest words in the vocabulary.  

Интересно использовать Word2Vec для поиска синонимов. Следующая функция вернет 'n' ближайших слов к заданному вводу. Чтобы найти их, вычисляем норму $|w_i - v|$, где $v$ - вектор, соответствующий нашему входному слову, а $w_i$ - кодировка $i$-го слова в словаре. Затем мы сортируем массив и возвращаем соответствующие индексы, используя 'argsort', и берем первые 'n' элементы списка, которые кодируют позиции ближайших слов в словаре.  

In [105]:
def close_words(x,n=5):
  vec = embedder(vectorizer(x))[0]
  top5 = np.linalg.norm(vectors-vec,axis=1).argsort()[:n]
  return [ vocab[x] for x in top5 ]

close_words('paris')

['paris', 'philippines', 'seoul', 'jakarta', 'zoo']

In [112]:
close_words('china')

['china', 'russia', 'pakistan', 'israel', 'turkey']

In [113]:
close_words('official')

['official', 'military', 'office', 'police', 'sources']

## Takeaway


## Еда на вынос

Using clever techniques such as CBoW, we can train Word2Vec model. You may also try to train skip-gram model that is trained to predict the neighboring word given the central one, and see how well it performs. 

Используя умные методы, такие как CBoW, мы можем обучить модель Word2Vec. Вы также можете попробовать обучить скип-граммовую модель, которая обучена предсказывать соседнее слово, заданное центральным, и посмотреть, насколько хорошо оно работает. 